Prediction based on trained model

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle


In [3]:
## This block is used to make predictions using our trained model on new data.
model=load_model("model.h5") # Loads your trained neural network from disk

## During training we have map Male->0 and Female->1, so in order to maintain same mapping we have import the encoder
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender=pickle.load(file)


with open('onehot_encoded_geo.pkl', 'rb') as file:
    onehot_encoded_geo=pickle.load(file)

# During training our model learns on the scaled value not raw ones, so we have to use the same scaler
with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)

In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 21,
    'Tenure': 3,
    'Balance': 75000,
    'NumOfProducts': 2,
    'HasCrCard': 0,
    'IsActiveMember': 0,
    'EstimatedSalary': 85000
}



In [12]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoded_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoded_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\DSA_DEV\Python_GenAI\Python_Lang\2_ANN+Project\venv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,21,3,75000,2,0,0,85000


In [14]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,21,3,75000,2,0,0,85000


In [15]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,21,3,75000,2,0,0,85000,1.0,0.0,0.0


In [16]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755, -1.7015827 , -0.69539349, -0.01764635,
         0.80843615, -1.54035103, -1.02583358, -0.26829875,  1.00150113,
        -0.57946723, -0.57638802]])

In [17]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 405ms/step


array([[0.01178344]], dtype=float32)

In [18]:
prediction_prob = prediction[0][0]
prediction_prob

0.011783439

In [19]:
if prediction_prob > 0.5:
    print("Customer is going to leave the bank.")
else:
    print("Customer is not going to leave the bank.")

Customer is not going to leave the bank.
